# 1. 模型

```
模型 (nn.Module)
|--模型创建
|   |--构建网络层
|   |   |--卷积层、池化层、激活函数层等
|   |--拼接网络层
|   |   |--LeNet、AlexNet、ResNet等
|--模型初始化
|   |--Xavier、Kaiming、均匀分布、正态分布等
```

## 1.1 模型创建 (以 `LeNet` 为例)

### 1). 构建子模块 `__init__()`
```python
def __init__(self, classes):
    super(LeNet, self).__init__()
    self.conv1 = nn.Conv2d(3, 6, 5)
    self.conv2 = nn.Conv2d(6, 16, 5)
    self.fc1 = nn.Linear(16*5*5, 120)
    self.fc2 = nn.Linear(120, 84)
    self.fc3 = nn.Linear(84, classes)
```

### 2). 拼接子模块 `forward()`
```python
def forward(self, x):
    out = F.relu(self.conv1(x))
    out = F.max_pool2d(out, 2)

    out = F.relu(self.conv2(x))
    out = F.max_pool2d(out, 2)

    out = out.view(out.size(0), -1)
    
    out = F.relu(self.fc1(out))
    out = F.relu(self.fc2(out))
    out = self.fc3(out)
    
    return out
```

### 综上，构建 `LeNet` 的整个过程：
```python
class LeNet(nn.Module):
    def __init__(self, classes):
        super(LeNet, self).__init__() ## super 实现父类函数的调用，这里指的是 LeNet 调用父类 nn.Module 的初始化函数。
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, classes)
    
    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)

        out = F.relu(self.conv2(x))
        out = F.max_pool2d(out, 2)

        out = out.view(out.size(0), -1)
        
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        
        return out
```

## 1.2 模型初始化

# 2. `nn.Module` 的详解

## 2.1 `torch.nn`

### 1). `nn.Module`
> 所有网络层的基类，管理网络属性。
>
> 关于 `nn.Module` 的总结：
> &emsp;&emsp; 1. 一个 `module` 可以包含多个子module；
>
> &emsp;&emsp; 2. 一个 `module` 相当于一个运算，必须实现 `forward()` 函数；
>
> &emsp;&emsp; 3. 每个 `module` 都有8个有序字典管理它的属性，重点关注 `modules` 和 `parameters`，管理模型和可学习参数；

**重点关注：**

**`parameters`:** `self._parameters = OrderDict()`，存储管理 `nn.Parameters` 类；

**`modules`：**`self._modules = OrderDict()`，存储管理 `nn.Modules` 类；

**`buffers`:** `self._buffers = OrderDict()`，存储管理缓冲属性，比如 BN层 中的均值、方差等；


### 2). `nn.Parameter`
> 张量(Tensor)子类，表示可学习参数，如 weight, bias。

### 3). `nn.functional`
函数具体实现，比如卷积、池化、激活函数等。

### 4). `nn.init`
参数初始化方法。

# 3. 模型容器 (Containers) 以及 `AlexNet` 的构建
```
模型容器 (Containers)
|--nn.Sequential
|   |-- 按顺序 包装多个网络层。
|   |-- 顺序性，各网络层之间严格按照顺序执行，常用语 block 构建。(整体上感觉 nn.Sequential 这个模型容器使用更方便)
|--nn.MuduleList
|   |-- 像 Python 的 list 一样包装多个网络层。
|   |-- 迭代性，常用于大量重复网络构建。
|--nn.ModuleDict
|   |-- 像 Python 的 dict 一样包装多个网络层。
|   |-- 索引性，常用语可选择的网络层。
```

## 3.1 Containers:

### 1). `nn.Sequential`

> 是 `nn.Module` 的容器，用于按顺序包装一组网络层。
>
> `nn.Sequential` 的特性：
> 
> &emsp;&emsp; 1. 顺序性：各网络层之间按照严格的顺序进行构建。
>
> &emsp;&emsp; 2. 自带 `forward()`：在自带的`forward()`中，通过for循环一次执行前向传播。


**示例：使用 `nn.Sequential` 构建 `LeNet`**

```python
## 方法1（网络层的名称使用默认的序号）;
class LeNetSequential(nn.Module):
    def __init__(self, classes):
        super(LeNetSequential, self).__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(3, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),

            nn.Conv2d(6, 16, 5),
            nn.ReLU()
            nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.classifier = nn.Sequential(
            nn.Linear(16*5*5, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, classes),
        )
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size[0], -1)
        x = self.classifier(x)
        return x




## 方法2（网络层的名称进行手动指定）
class LeNetSequentialOrderDict(nn.Module):
    def __init__(self, classes):
        super(LeNetSequentialOrderDict, self).__init__()
        
        self.features = nn.Sequential(OrderDict({
            "conv1": nn.Conv2d(3, 6, 5),
            "relu1": nn.ReLU(inplace=True),
            "pool1": nn.MaxPool2d(kernel_size=2, stride=2),

            "conv2": nn.Conv2d(6, 16, 5),
            "relu2": nn.ReLU(inplace=True),
            "pool2": nn.MaxPool2d(kernel_size=2, stride=2),
        }))

        self.classifier = nn.Sequential(OrderDict({
            "fc1": nn.Linear(16*5*5, 120),
            "relu3": nn.ReLU(inplace=True),
            "fc2": nn.Linear(120, 84),
            "relu4": nn.ReLU(inplace=True),
            "fc3": nn.Linear(84, classes),
        }))
    
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size[0], -1)
        x = self.classifier(x)
        return x
```


### 2). `nn.ModuleList`
> 也是 `nn.Module` 的容器，用于包装一组网络层，并以迭代的方式调用网络层。
> 
> 主要方法：
> 
> `append()`：在 ModuleList 后面添加网络层；
>
> `extend()`: 拼接两个 ModuleList；
>
> `insert()`: 指定在 ModuleList 中特定位置插入网络层；


**示例：用 `nn.ModuleList` 实现20个全连接层（每层有10个神经元）构成的网络**

```python
class FCModuleList(nn.Module):
    def __init__(self):
        super(FCModuleList, self).__init__()
        
        self.linears = nn.ModuleList([nn.Linear(10,10) for i in range(20)]) ## 用列表生成式进行构造

    def forward(self, x):
        for i, linear in enumerate(self.linears):
            x = linear(x)
        return x

```

### 3. `nn.ModuleDict`
> 也是 `nn.Module` 的容器，用于包装一组网络层，以索引的方式调用网络层。
>
> 主要方法：
>
> `clear()`: 清空 ModuleDict
>
> `items()`: 返回可迭代的 key-value
>
> `keys()`: 返回字典的 key
>
> `values()`: 返回字典的 value
>
> `pop()`: 返回一对 key-value，并从字典中删除


**示例: Conv + Relu 的组合**
```python
class ModuleDict(nn.Module):
    def __init__(self):
        super(ModuleDict, self).__init__()
        
        self.choices = nn.ModuleDict({
            "conv": nn.Conv2d(10, 10, 3),
            "pool": nn.MaxPool2d(3)
        })

        self.activations = nn.ModuleDict({
            "relu": nn.ReLU(),
            "prelu": nn.PReLU()
        })

    def forward(self, choice, act):
        x = self.choices[choice](x)
        x = self.activations[act](x)
        return x
```

## 3.2 `AlexNet`的构建
> `AlexNet`的特点：
>
> 1. 采用 ReLU 代替饱和激活函数(比如 Sigmoid)，减轻梯度消失；
>
> 2. 采用 LRN (Local Response Normalization)，对数据归一化，减轻梯度消失；
>
> 3. 采用 Dropout 提高全连接层的鲁棒性，增强网络的泛化能力；
>
> 4. 采用 Data Augmentation；

**`AlexNet`的具体实现**
 
```python
## 来自于 Pytorch 官方提供的 AlexNet 实现 (torchvision.models.AlexNet())
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 1000, dropout: float = 0.5) -> None:
        super().__init__()
        _log_api_usage_once(self)
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x
```


# 神经网络中常用的层

## 1. 卷积层

> 卷积运算：卷积核在图像上滑动，相应位置上进行乘加（卷积核与图像对应位置进行相乘，然后在把结果相加）；
> 
> 卷积核：也成滤波器、过滤器。
>
> 卷积维度：一般情况下，卷积核在几个维度上滑动，就是几维卷积。

### `nn.Conv2d`
功能：对多个二维信号进行二维卷积。

```python
nn.Conv2d(
    in_channels,  ## 输入通道数
    out_channels, ## 输出通道数，等价于卷积核个数
    kernel_size,  ## 卷积核尺寸
    stride=1,     ## 步长 (1 : [1,2,3] -> [2,3,4]; 2: [1,2,3] -> [3,4,5])
    padding=0,    ## 填充个数，保持输入输出的图片的尺寸不变
    dilation=1,   ## 空洞卷积大小 ("带孔"的卷积核，常用于图像分割任务，提升感受野)
    groups=1,     ## 设置分组卷积的组数 (常用于模型的轻量化，比如 AlexNet的原始实现)
    bias=True,    ## 偏置
    padding_mode="zeros"
)
```
图像经过卷积之后的尺寸是如何计算的：

（简化版）`Out_size = (In_size - Kernel_size) / stride + 1`
（完整版）`H_out = (H_in + 2 x padding - dilation x (kernel_size-1) -1)/stride + 1`

### 转置卷积
也称为反卷积 (Deconvolution) 和部分跨越卷积 (Fractionally-strided Convolution)，用于对图像进行上采样（图像分割中常用）。

简单来说，转置卷积就是将小尺寸图像经过卷积之后得到一个更大尺寸的图像。

```python
nn.ConvTranspose2d(
    in_channels,    ## 输入通道数
    out_channels,   ## 输出通道数
    kernel_size,    ## 卷积核尺寸
    stride=1,       ## 步长
    padding=0,      ## 填充个数
    output_padding=0,
    groups=1,       ## 分组卷积设置
    bias=True,      ## 偏置
    dilation=1,     ## 空洞卷积大小
    padding_mode="zeros"
)
```

## 池化层 `Pooling Layer`
作用：对信号进行 "收集" 并 "总结"。（剔除冗余信息，减少后续计算量；）（大尺寸 --> 小尺寸）

"收集"：多变少；

"总结"：最大值/平均值；



### `nn.MaxPool2d`
对二维信号进行最大池化；

```python
## MaxPool2d
nn.MaxPool2d(
    kernel_size, ## 池化核的尺寸
    stride=None, ## 步长
    padding=0,   ## 填充个数
    dilation=1,  ## 池化核之间间隔的大小
    return_indices=False, ## 记录池化像素索引（这个在反池化过程中会用到）
    ceil_mode=False  ## 尺寸向下取整
)
```

### `nn.AvgPool2d`
对二维信号进行平均值池化；


```python
## AvgPool2d
nn.AvgPool2d(
    kernel_size,
    stride=None,
    padding=0,
    ceil_mode=False,
    count_include_pad=True, ## 填充值用于计算
    divisor_override=None  ## 除法因子
)
```

## 反池化
对二维像素进行最大值池化上采样（小尺寸 --> 大尺寸）

### `nn.MaxUnpool2d`
对二维信号进行最大池化上采样。

```python
## 最大反池化
nn.MaxUnpool2d(
    kernel_size,
    stride=None,
    padding=0
)

## 在前向传播时，需要传入索引indices
forward(self, input, indices, output_size=None)
```

## 线性层 (全连接层)
每个神经元与上一层所有神经元相连实现对前一层的线性组合、线性变换。

### `nn.Linear`
对一维信号（向量）进行线性组合。

```python
nn.Linear(
    in_features,  ## 输入结点数
    out_features, ## 输出结点数
    bias=True ## 是否需要偏置
)
```

## 激活函数层
对特征进行非线性变换，使得多层神经网络有了“深度”的含义。

如果没有激活层，那么多层线性层等价于一层线性层！

### `nn.Sigmoid`
“S形曲线”

公式：y = 1/(1+e**(-x))

梯度公式：y' = y*(1-y)

特性：

> 输出值在(0,1)，符合概率分布；
>
> 导数范围是[0, 0.25]，容易导致梯度消失；
> 
> 输出为非0均值，破坏数据分布。

### `nn.tanh`
“双曲正切函数”

特性：
> 输出值在 (-1,1)，数据符合0均值；
>
> 导数范围是(0,1)，易导致梯度消失；

### `nn.ReLU`
针对 `Sigmoid`和`tanh`存在的梯度爆炸爆炸问题，提出了`ReLU`的方法缓解梯度爆炸问题。

计算公式：y=max(0,x)

特性：
> 输出值均为正数，负半轴导致死神经元；
>
> 导数是1，缓解梯度消失，但易引发梯度爆炸。

### 针对 `ReLU` 负半轴输出为0的情况，有一些 `ReLU`的变种：
**`nn.LeakyReLU`**

> 在负半轴增加了一个很小的斜率；(可以通过`negative_slope` 参数来设置负半轴斜率)

**`nn.PReLU`**

> 斜率为可学习的参数；(可以通过 `init` 参数设置可学习斜率)

**`nn.RReLU`**

> 斜率是随机的，每次都从一个均匀分布中去随机采样；(可以通过`lower`和`upper`设置均匀分布的下限和上限)

# 权重初始化

## `Xavier` 初始化
针对饱和激活函数，如 Sigmoid, Tanh (保持数据尺度在恰当范围，通常方差为1)

`nn.init.xavier_uniform_(m.weight.data)`

## `Kaiming` 初始化
针对非饱和激活函数，如 ReLU及其变种 (保持数据尺度在恰当范围，通常方差为1)

`nn.init.kaiming_normal_(m.weight.data)`

## 10种初始化方法：
具体选取哪种方法要具体问题具体分析，最终的目的是为了维持网络层的输出值不能太大，也不能太小，尽量保持每一层额度输出值方差为1

```
1. Xavier 均匀分布
2. Xavier 标准正态分布

3. Kaiming 均匀分布
4. Kaiming 标准正态分布

5. 均匀分布
6. 正态分布
7. 常数分布

8. 正交矩阵初始化
9. 单位矩阵初始化
10. 稀疏矩阵初始化
```

## 计算激活函数的方差变化尺度：
方差变化尺度 = 输入数据的方差 / 激活函数后输出的数据的方差

```python
nn.init.calculate_gain(nonlinearity,  ## 激活函数名称，比如 Sigmoid, tanh, ReLU
                       param=None ## 激活函数的参数，比如 LeakyReLU 的 negative_slpop
)
```


# 损失函数
损失函数：衡量模型输出与真实标签的差异。

损失函数 (Loss Function): `Loss = f(y^, y)` (计算一个样本的损失)

代价函数 (Cost Function): `Cost = mean(f(y^, y))` (计算整个数据集的平均损失)

目标函数 (Objective Function): `Obj = Cost + Regularization(正则项)` (训练模型的目的就是得到一个目标函数)

## `nn.CrossEntropyLoss`
功能：`nn.LogSoftmax()` 与 `nn.NLLLoss()` 结合，进行交叉熵计算。

交叉熵：衡量两个分布之间的差异。

```python
nn.CrossEntropyLoss(
    weight=None, ## 各类别的loss设置权值
    size_average=None, ## 该参数后续回被去掉
    ignore_index=-100, ## 忽略某个类别
    reduce=None, ## 该参数后续回被去掉
    reduction="mean"  ## 计算模式，可为 none(逐个元素计算)/sum(所有元素求和，返回标量)/mean(加权平均，返回标量)
)
```

**举例：**
```python
inputs = torch.tensor([1,2],[1,3],[1,3], dtype=torch.float) ## inputs 的数据类型是 float
target = torch.tensor([0,1,1], dtype=torch.long) ## target 的数据类型是 long

loss_func = nn.CrossEntropyLoss(weight=None, reduction="none")
loss_none = loss_func(inputs, target)
```

```python
weights = torch.tensor([1,2], dtype=torch.float) ## 每个类别都要设置 weight，组成一个 list
## 如果有10个类别，那这里 weights 中list的长度是10，需要有10个元素。
## 如果 reduction = "mean" ，那么weights不用太关注它的尺度，只需要关注各类别之间的比例即可；

loss_func = nn.CrossEntropyLoss(weights=weights, reduction="None")
loss_none = loss_func(inputs, target)
```

## `nn.NLLLoss`
功能：实现负对数似然函数中的**负号功能**。(就是将输入值取负号)

```python
nn.NLLLoss(
    weight=None, ## 各类别的loss的权值
    size_average=None, 
    ignore_index=-100,
    reduction=None,
    reduction="mean" ## 计算模式
)
```

## `nn.BCELoss`
功能：二分类交叉熵。

注意：输入值的取值在[0,1]之间。

```python
nn.BCELoss(
    weight=None, ## 权重设置
    size_average=None,
    reduce=None,
    reduction="mean" ## 计算模式
)
```

```python
inputs = torch.tensor([[1,2],[2,2],[3,4],[4,5]], dtype=torch.float) ## 这里的 inputs 和 target 都是 float类型
target = torch.tensor([[1,0],[1,0],[0,1],[0,1]], dtype=torch.float)

inputs = torch.sigmoid(inputs) ## 需要用 sigmoid 将 inputs 缩放到 0-1 之间
loss_func = nn.BCELoss(weights=None, reduction="none")

loss_none = loss_func(inputs, target)
```

## `nn.BCEWithLogitsLoss`
功能：结合 Sigmoid 与二分类交叉熵。

注意：网络最后不加 sigmoid 函数。

```python
nn.BCEWithLogitsLoss(
    weights=None,
    size_average=None,
    reduce=None,
    reduction="mean", ## 计算模式
    pos_weight=None ## 正样本的权值 (当 pos:neg = 1:3 时，可以将pos_weight设置为3)，需要是一个 tensor
)
```

## `nn.L1Loss`
功能：计算 inputs 和 target 之差的绝对值。

```python
nn.L1Loss(
    size_average=None,
    reduce=None,
    reduction="mean"
)
```

## `nn.MSELoss`
功能：计算 inputs 和 target 之差的平方。

```python
nn.MSELoss(
    size_average=None,
    reduce=None,
    reduction="mean" ## 计算模式
)
```

## `nn.SmoothL1Loss`
功能：平滑的 L1Loss (减轻离群点对模型的影响)

```python
nn.SmoothL1Loss(
    size_average=None,
    reduce=None,
    reduction="mean"
)
```

## `PoissonNLLLoss`
功能：泊松分布的负对数似然损失函数。(输出符合泊松分布的时候可以使用)

```python
nn.PoissonNLLLoss(
    log_input=True, ## 输入是否为对数形式，决定计算公式
    full=False, ## 计算所有的 loss，默认为False
    size_average=None,
    eps=1e-8, ## 修正项，避免log(input)为nan
    reduce=None,
    reduction="mean"
)
```

## `nn.KLDivLoss`
功能：计算 KLD，KL散度，相对熵。

注意：需提前将输入进行取log_probabilities，比如通过 `nn.logsoftmax()`

```python
nn.KLDivLoss(
    size_average=None,
    reduce=None,
    reduction="mean" ## 计算模式，包括 none/sum/mean/batchmean
)
```

## `nn.MarginRankingLoss`
功能：计算两个向量之间的相似度，用于排序任务。

注意：该方法计算两组数据之间的差异，返回一个 `n*n` 的 loss 矩阵。

```python
nn.MarginRankingLoss(
    margin=0.0, ## 边界值，x1与x2之间的差异值
    size_average=None,
    reduce=None,
    reduction="mean" # 计算模式，包括 none/sum/mean
)
```

## `nn.MultiLabelMarginLoss`
功能：多标签边界损失函数。

```python
nn.MultiLabelMarginLoss(
    size_average=None,
    reduce=None,
    reduction="mean" ## 计算模式，包括 none/sum/mean
)
```

## `nn.SoftMarginLoss`
功能：计算二分类的 logistic 损失。

```python
nn.SoftMarginLoss(
    size_average=None,
    reduce=None,
    reduction="mean"
)
```

## `nn.MultiLabelSoftMarginLoss`
功能： `SoftMarginLoss` 多标签版本。

```python
nn.MultiLabelSoftMarginLoss(
    weight=None,
    size_average=None,
    reduce=None,
    reduction="mean"
)
```

## `nn.MultiMarginLoss`
功能：计算多酚类的折页损失。

```python
nn.MultiMarginLoss(
    p=1, ## 可选1或2
    margin=1.0, ## 边界值
    weight=None,
    size_average=None,
    reduce=None,
    reduction="mean"
)
```

## `nn.TripletMarginLoss`
功能：计算三元损失，人脸验证中常用。

```python
nn.TripletMarginLoss(
    margin=1.0, ## 边界值
    p=2.0, ## 范数的阶，默认为2
    eps=1e-6,
    swap=False,
    size_average=None,
    reduce=None,
    reduction="mean"
)
```

## `nn.HingeEmbeddingLoss`
功能：计算两个输入的相似性，常用于 非线性embedding 和半监督学习。

注意：输入x应为两个输入之差的绝对值。

```python
nn.HingeEmbeddingLoss(
    margin=1.0, ## 边界值
    size_average=None,
    reduce=None,
    reduction="mean"
)
```

## `nn.CosineEmbeddingLoss`
功能：采用cos相似度计算两个输入的相似性。

```python
nn.CosineEmbeddingLoss(
    margin=0.0, ## 可取值[-1,1]，推荐为[0,0.5]
    size_average=None,
    reduce=None,
    reduction="mean"
)
```

## `nn.CTCLoss`
功能：计算 CTC 损失，解决时序类数据的分类。

```python
nn.CTCLoss(
    blank=0, ## blank label
    reduction="mean",
    zero_infinity=False ## 无穷大的值或者梯度置0
)
```

# 优化器 (`optimizer`)
管理并更新模型中可学习参数的值，使模型输出更接近真实标签。

**基本属性：**

`defaults`：优化器超参数。

`state`：参数的缓存，比如 momentum 的缓存。

`param_groups`：管理的参数组。

`_step_count`：记录更新次数，调整学习率的时候使用。


**基本方法：**

`zero_grad()`：清除所管理参数的梯度。（Pytorch中的张量梯度不会自动清零）

`step()`：执行一步梯度更新。

`add_param_group()`：添加参数组。

(下面两个主要用于模型的断点恢复训练，每个一段时间保存优化器状态信息)

`state_dict()`：获取优化器当前状态信息字典。

`load_state_dict()`：将状态信息字典加载到优化器中。

## 学习率:

梯度下降：`Wi+1 = Wi - LR * g(Wi)`

## 优化器：

`Momentum (动量，冲量)`：结合当前梯度与上一次更新信息，用于当前更新（通常设置为 0.9）。
`vi = m * vi-1 + g(wi)`

`wi+1 = wi - lr * vi`

## 1. `torch.optim.SGD`

```python
optim.SGD(
    params, ## 管理的参数组
    lr,     ## 学习率
    momentum=0, ## 动量系数
    dampening=0, 
    weight_decay=0, ## L2 正则化系数
    nesterov=False ## 是否采用NAG
)
```

## 2. pytorch 提供的多种优化器
|优化器|用途|
|--|--|
|`optim.SGD`|随机梯度下降法|
|`optim.Adagrad`|自适应学习率梯度下降法|
|`optim.RMSprop`|Adagrad的改进|
|`optim.Adadelta`|Adagrad的改进|
|`optim.Adam`|RMSprop结合Momentum|
|`optim.Adamax`|Adam增加学习率上限|
|`optim.SparseAdam`|稀疏版的Adam|
|`optim.ASGD`|随机平均梯度下降|
|`optim.Rprop`|弹性反向传播 (full-batch时使用，但是mini-batch中不可用)|
|`optim.LBFGS`|BFGS的改进|
